In [15]:
import tensorflow as tf
import pickle
import pandas as pd

In [16]:
# Loading the files
model=tf.keras.models.load_model("model.h5")
labelencoder=pickle.load(open('labelencoder.pkl','rb'))
onehotencoder=pickle.load(open('onehotencoder.pkl','rb'))
sc=pickle.load(open('scaler.pkl','rb'))


In [17]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [34]:
# one hot encode the categorical data
geo_encoded = onehotencoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehotencoder.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [19]:
input=pd.DataFrame([input_data])

In [20]:
input.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [21]:
# converting gender using label encoder
input["Gender"]=labelencoder.transform(input["Gender"])


In [22]:
input.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [35]:
## concatination one hot encoded 
input_df=pd.concat([input.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [37]:
# scaling the data
input_df=sc.transform(input_df)

In [40]:
predictionprob=model.predict(input_df)[0][0]
print(predictionprob)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.039305083


In [42]:
if predictionprob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
